### Original Data Sources (For Reference and Extension Work)
1. **Mapping Prejudice Project**
   - URL: https://mappingprejudice.umn.edu/
   - Contains: Historical covenant data for Hennepin County
   - Format: GIS data showing covenant locations and details

2. **Census TIGER/LINE Files**
   - URL: https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2024&layergroup=Census+Tracts
   - Contains: Census tract boundaries
   - Format: Geographic shapefiles

3. **HMDA Data Browser**
   - URL: https://ffiec.cfpb.gov/data-browser/data/2023?category=states&items=MN
   - Contains: Modern mortgage lending data
   - Format: CSV files with lending information

### In this block we filter the Minnesota tract data from the Census database

[https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2024&layergroup=Census+Tracts](https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2024&layergroup=Census+Tracts)

The county code is hard coded in this block, so you will need to change it for other counties.

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
import matplotlib.pyplot as plt
# import seaborn as sns


# Read the full state file
mn_tracts = gpd.read_file('tl_2024_27_tract.zip')

# Filter for Hennepin County (FIPS code 053)
# hennepin_tracts = mn_tracts[mn_tracts['COUNTYFP'] == '053']

# Filter for Ramsey County (FIPS code 123)
ramsey_tracts = mn_tracts[mn_tracts['COUNTYFP'] == '123']

# # Save just Hennepin County if you want
# hennepin_tracts.to_file('hennepin_tracts.shp')

In [6]:
hennepin_tracts.head(2)

,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
79,27,053,023401,27053023401,1400000US27053023401,234.01,Census Tract 234.01,G5020,S,2137560,5545,+44.9125314,-093.4096872,"POLYGON ((-93.42229 44.91246, -93.42187 44.912..."
80,27,053,026724,27053026724,1400000US27053026724,267.24,Census Tract 267.24,G5020,S,8119115,351531,+45.1014129,-093.4147141,"POLYGON ((-93.46024 45.10454, -93.45989 45.104..."


In [2]:
ramsey_tracts.head(2)

,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
124,27,123,040102,27123040102,1400000US27123040102,401.02,Census Tract 401.02,G5020,S,5810969,3545673,+45.1150726,-093.0093805,"POLYGON ((-93.03584 45.09738, -93.03317 45.097..."
125,27,123,040804,27123040804,1400000US27123040804,408.04,Census Tract 408.04,G5020,S,3309782,1257720,+45.0454768,-093.1591990,"POLYGON ((-93.17755 45.04262, -93.17753 45.042..."


### Mortgage data from 2023

We now reading the mortgage data for Minnesota in 2023.

[https://ffiec.cfpb.gov/data-browser/data/2023?category=states&items=MN](https://ffiec.cfpb.gov/data-browser/data/2023?category=states&items=MN)

Data is available back to 2018.

In [3]:
mortgage_data = pd.read_csv('state_MN.csv')
mheaders = mortgage_data.columns
mheaders

/var/folders/42/5jr766g57bd9bxgy6m30dz880000gn/T/ipykernel_19089/83746537.py:1: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,44) have mixed types. Specify dtype option on import or set low_memory=False.
  mortgage_data = pd.read_csv('state_MN.csv')


Index(['activity_year', 'lei', 'derived_msa-md', 'state_code', 'county_code',
       'census_tract', 'conforming_loan_limit', 'derived_loan_product_type',
       'derived_dwelling_category', 'derived_ethnicity', 'derived_race',
       'derived_sex', 'action_taken', 'purchaser_type', 'preapproval',
       'loan_type', 'loan_purpose', 'lien_status', 'reverse_mortgage',
       'open-end_line_of_credit', 'business_or_commercial_purpose',
       'loan_amount', 'loan_to_value_ratio', 'interest_rate', 'rate_spread',
       'hoepa_status', 'total_loan_costs', 'total_points_and_fees',
       'origination_charges', 'discount_points', 'lender_credits', 'loan_term',
       'prepayment_penalty_term', 'intro_rate_period', 'negative_amortization',
       'interest_only_payment', 'balloon_payment',
       'other_nonamortizing_features', 'property_value', 'construction_method',
       'occupancy_type', 'manufactured_home_secured_property_type',
       'manufactured_home_land_property_interest', 'total_

In [4]:
# data set size
mortgage_data.shape

(174738, 99)

### We will filter the loans to only be mortgages.

[https://ffiec.cfpb.gov/documentation/publications/loan-level-datasets/lar-data-fields](https://ffiec.cfpb.gov/documentation/publications/loan-level-datasets/lar-data-fields)

This is loan_purpose == 1

In [5]:
# filter by loan_purpose = 1 (home purchase)
mortgage_data = mortgage_data[mortgage_data['loan_purpose'] == 1]

In [7]:
covenent = pd.read_csv('covenants-mn-ramsey-county.csv')
cheaders = covenent.columns
cheaders

Index(['db_id', 'workflow', 'cnty_name', 'cnty_fips', 'doc_num', 'deed_year',
       'deed_date', 'exec_date', 'cov_text', 'seller', 'buyer', 'street_add',
       'city', 'state', 'zip_code', 'add_cov', 'block_cov', 'lot_cov',
       'cnty_pin', 'add_mod', 'block_mod', 'lot_mod', 'ph_dsc_mod',
       'join_strgs', 'geocd_addr', 'geocd_dist', 'cov_type', 'match_type',
       'manual_cx', 'dt_updated', 'zn_subj_id', 'zn_dt_ret', 'image_ids',
       'med_score', 'plat_dbid', 'subd_dbid'],
      dtype='object')

In [8]:
geocode_data = pd.read_csv('ramsey-geocode-results.csv')

In [9]:
geocode_data.head(2)

,db_id,workflow,cnty_name,cnty_fips,doc_num,deed_year,deed_date,exec_date,cov_text,seller,...,dt_updated,zn_subj_id,zn_dt_ret,image_ids,med_score,plat_dbid,subd_dbid,full_add,latitude,longitude
0,121778,1,Ramsey,27123,62278,1923,7/30/23,NaN,The party of the second part hereby agrees tha...,Oric O. & Louise F. Whited,...,2024-02-13 20:16:09.492556+00:00,NaN,NaN,",,",NaN,1635.0,NaN,"1641 JUNO AVE, ST PAUL, MN, 55116",-93.169652,44.926151
1,10042,1,Ramsey,27123,238504,1946,12/6/46,NaN,"No lot or parcel shall ever be sold, conveyed,...","Siems Bros, Inc.",...,2024-02-13 20:07:42.878392+00:00,NaN,NaN,",,",NaN,2211.0,NaN,"3500 GLENARDEN RD, ARDEN HILLS, MN, 55112",-93.158148,45.047015


In [10]:
covenent.head(2)

,db_id,workflow,cnty_name,cnty_fips,doc_num,deed_year,deed_date,exec_date,cov_text,seller,...,cov_type,match_type,manual_cx,dt_updated,zn_subj_id,zn_dt_ret,image_ids,med_score,plat_dbid,subd_dbid
0,121778,1,Ramsey,27123,62278,1923,1923-07-30,NaN,The party of the second part hereby agrees tha...,Oric O. & Louise F. Whited,...,manual,Public submission (single property),True,2024-02-13 20:16:09.492556+00:00,NaN,NaN,",,",NaN,1635.0,NaN
1,10042,1,Ramsey,27123,238504,1946,1946-12-06,NaN,"No lot or parcel shall ever be sold, conveyed,...","Siems Bros, Inc.",...,manual,Something else,True,2024-02-13 20:07:42.878392+00:00,NaN,NaN,",,",NaN,2211.0,NaN


In [11]:
covenent[covenent['street_add'] == '1641 JUNO AVE']

,db_id,workflow,cnty_name,cnty_fips,doc_num,deed_year,deed_date,exec_date,cov_text,seller,...,cov_type,match_type,manual_cx,dt_updated,zn_subj_id,zn_dt_ret,image_ids,med_score,plat_dbid,subd_dbid
0,121778,1,Ramsey,27123,62278,1923,1923-07-30,NaN,The party of the second part hereby agrees tha...,Oric O. & Louise F. Whited,...,manual,Public submission (single property),True,2024-02-13 20:16:09.492556+00:00,NaN,NaN,",,",NaN,1635.0,NaN


In [13]:
covenent[['street_add', 'city', 'state', 'zip_code']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5457 entries, 0 to 5456
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   street_add  5457 non-null   object
 1   city        5457 non-null   object
 2   state       5457 non-null   object
 3   zip_code    5457 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 170.7+ KB


### We need to make our data sets a bit more manageable before merging

In [ ]:
covenent_bs = pd.read_csv('extradata.csv')

In [15]:
# covenant_slim = covenent[['FID', 'Racial_Res', 'Date_Deed', 'X', 'Y']]
covenant_slim = covenent_bs[['db_id', 'deed_date', 'longitude', 'latitude']]

In [16]:
mortgage_slim = mortgage_data[[
    'activity_year',
    'census_tract',
    'derived_race',
    'action_taken',
    'loan_amount',
    'property_value',
    'income',
    'interest_rate',
    'tract_minority_population_percent',
    'tract_to_msa_income_percentage',
    'denial_reason-1'
]]

### Merge the two data sets

Merging the two datasets is tricky because this is geographic data.  One data set uses Lat and Long.  The other is set up to use census data tracts.  To put this together, we need to convert Lat and Long into UTM coordinates and determine what is inside the tract boundary.

In [46]:
covenant_gdf.head()

,db_id,deed_date,longitude,latitude,geometry
0,121778,7/30/23,44.926151,-93.169652,POINT (Infinity Infinity)
1,10042,12/6/46,45.047015,-93.158148,POINT (Infinity Infinity)
2,129679,3/31/41,0.000000,0.000000,POINT (Infinity Infinity)
3,129717,3/14/40,45.102398,-93.224932,POINT (Infinity Infinity)
4,129748,1/17/42,45.104044,-93.225974,POINT (Infinity Infinity)


In [47]:
ramsey_tracts.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
124,27,123,040102,27123040102,1400000US27123040102,401.02,Census Tract 401.02,G5020,S,5810969,3545673,+45.1150726,-093.0093805,"POLYGON ((-93.03584 45.09738, -93.03317 45.097..."
125,27,123,040804,27123040804,1400000US27123040804,408.04,Census Tract 408.04,G5020,S,3309782,1257720,+45.0454768,-093.1591990,"POLYGON ((-93.17755 45.04262, -93.17753 45.042..."
126,27,123,040101,27123040101,1400000US27123040101,401.01,Census Tract 401.01,G5020,S,7508445,910515,+45.1070109,-093.0401804,"POLYGON ((-93.05678 45.12401, -93.05642 45.124..."
127,27,123,040708,27123040708,1400000US27123040708,407.08,Census Tract 407.08,G5020,S,2940007,678592,+45.0735909,-093.1325218,"POLYGON ((-93.14738 45.07939, -93.14613 45.079..."
128,27,123,040709,27123040709,1400000US27123040709,407.09,Census Tract 407.09,G5020,S,3242565,107945,+45.0660386,-093.1162382,"POLYGON ((-93.14722 45.0647, -93.1451 45.06471..."


In [48]:
covenant_slim.head()

,db_id,deed_date,longitude,latitude
0,121778,7/30/23,44.926151,-93.169652
1,10042,12/6/46,45.047015,-93.158148
2,129679,3/31/41,0.000000,0.000000
3,129717,3/14/40,45.102398,-93.224932
4,129748,1/17/42,45.104044,-93.225974


In [ ]:
# Convert covenant data to GeoDataFrame with correct initial CRS
geometry = [Point(xy) for xy in zip(covenant_slim['longitude'], covenant_slim['latitude'])]
covenant_gdf = gpd.GeoDataFrame(covenant_slim, geometry=geometry)
covenant_gdf.set_crs(epsg=4326, inplace=True)  # WGS84 decimal degrees

print("Original covenant points:", len(covenant_gdf))

# Transform to match the tracts CRS (UTM Zone 15N)
covenant_gdf = covenant_gdf.to_crs(epsg=26915)

# Quick check of transformed coordinates
print("\nTransformed coordinate ranges:")
print("X range:", covenant_gdf.geometry.x.min(), "to", covenant_gdf.geometry.x.max())
print("Y range:", covenant_gdf.geometry.y.min(), "to", covenant_gdf.geometry.y.max())

# Verify these points fall within tract bounds
tract_bounds = ramsey_tracts.total_bounds
print("\nTract bounds:")
print("X range:", tract_bounds[0], "to", tract_bounds[2])
print("Y range:", tract_bounds[1], "to", tract_bounds[3])

# Perform spatial join with transformed coordinates
covenants_with_tracts = gpd.sjoin(covenant_gdf, ramsey_tracts, how='left', predicate='within')

# See how many points matched to tracts
print("\nPoints that matched to tracts:", len(covenants_with_tracts))
print("Points with null tract assignments:", covenants_with_tracts['TRACTCE'].isna().sum())

# Count covenants per tract
covenants_per_tract = covenants_with_tracts.dropna(subset=['TRACTCE']).groupby('TRACTCE').size().reset_index(name='covenant_count')

print("\nTract Summary:")
print("Number of tracts with covenants:", len(covenants_per_tract))
if len(covenants_per_tract) > 0:
    print("\nCovenant counts per tract:")
    print(covenants_per_tract['covenant_count'].describe())

Original covenant points: 5457

Transformed coordinate ranges:
X range: inf to inf
Y range: inf to inf

Tract bounds:
X range: -93.228119 to -92.984192
Y range: 44.887399 to 45.124533

Points that matched to tracts: 5457
Points with null tract assignments: 5457

Tract Summary:
Number of tracts with covenants: 0


/var/folders/42/5jr766g57bd9bxgy6m30dz880000gn/T/ipykernel_19089/2473239917.py:23: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:26915
Right CRS: EPSG:4269

  covenants_with_tracts = gpd.sjoin(covenant_gdf, ramsey_tracts, how='left', predicate='within')


In [18]:
def format_census_tract(value):
    try:
        # First convert to string to handle any type
        tract_str = str(value)
        # Remove any decimal points and trailing zeros
        tract_str = tract_str.split('.')[0]
        return tract_str
    except Exception as e:
        print(f"Error processing value: {value}, Type: {type(value)}")
        raise e

# Create a clean copy and try the conversion
mortgage_clean = mortgage_slim.copy()
mortgage_clean['census_tract'] = mortgage_clean['census_tract'].astype(str)
mortgage_clean['census_tract'] = mortgage_clean['census_tract'].apply(format_census_tract)

# Let's see what we got
print("\nAfter cleaning:")
print(mortgage_clean['census_tract'].head())


After cleaning:
0    27037060737
2    27141030504
3    27023950400
4    27171100101
5    27159480100
Name: census_tract, dtype: object


In [43]:
covenants_with_tracts

,db_id,deed_date,longitude,latitude,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,census_tract
0,121778,7/30/23,44.926151,-93.169652,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10042,12/6/46,45.047015,-93.158148,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,129679,3/31/41,0.000000,0.000000,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,129717,3/14/40,45.102398,-93.224932,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,129748,1/17/42,45.104044,-93.225974,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5452,64527,12/29/19,NaN,NaN,POINT EMPTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5453,91215,9/11/37,NaN,NaN,POINT EMPTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5454,58744,9/23/47,NaN,NaN,POINT EMPTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5455,93492,7/1/22,NaN,NaN,POINT EMPTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
covenants_with_tracts.groupby('census_tract')

In [44]:
mortgage_clean

,activity_year,census_tract,derived_race,action_taken,loan_amount,property_value,income,interest_rate,tract_minority_population_percent,tract_to_msa_income_percentage,denial_reason-1
0,2023,27037060737,Race Not Available,6,185000.0,195000,NaN,6.125,40.37,73.48,10
2,2023,27141030504,Race Not Available,6,375000.0,385000,NaN,6.625,10.64,121.26,10
3,2023,27023950400,Race Not Available,6,105000.0,105000,NaN,6.125,8.92,90.13,10
4,2023,27171100101,Race Not Available,6,285000.0,285000,NaN,6.75,13.27,104.37,10
5,2023,27159480100,Race Not Available,6,175000.0,175000,NaN,6.875,5.87,82.22,10
...,...,...,...,...,...,...,...,...,...,...,...
174732,2023,27009020100,Race Not Available,6,255000.0,265000,NaN,6.125,4.88,106.18,10
174733,2023,27067780500,Race Not Available,6,185000.0,335000,NaN,6.125,46.58,74.68,10
174734,2023,27149480200,Race Not Available,6,65000.0,75000,NaN,6.75,20.14,118.83,10
174736,2023,27059130501,Race Not Available,6,545000.0,525000,NaN,5.0,6.84,90.07,10


In [19]:
# Clean mortgage data - first ensure we have strings
mortgage_clean = mortgage_slim.copy()
mortgage_clean['census_tract'] = mortgage_clean['census_tract'].astype(str)
mortgage_clean['census_tract'] = mortgage_clean['census_tract'].str.replace('.0', '')

# Clean mortgage data - first ensure we have strings
mortgage_clean = mortgage_slim.copy()
mortgage_clean['census_tract'] = mortgage_clean['census_tract'].astype(str)
mortgage_clean['census_tract'] = mortgage_clean['census_tract'].str.replace('.0', '')



# First format the covenant census tract IDs
covenants_with_tracts['census_tract'] = (
    covenants_with_tracts['STATEFP'] + 
    covenants_with_tracts['COUNTYFP'] + 
    covenants_with_tracts['TRACTCE']
)

# Create the per-tract counts
covenants_per_tract = covenants_with_tracts.groupby('census_tract').size().reset_index(name='covenant_count')

# Clean the mortgage data
mortgage_clean = mortgage_slim.copy()
mortgage_clean['census_tract'] = mortgage_clean['census_tract'].astype(str).str.replace('.0', '')

# Verify the formats match
print("Mortgage data examples:")
print(mortgage_clean['census_tract'].head())
print("\nCovenant data examples:")
print(covenants_per_tract['census_tract'].head())






# Merge the datasets
combined_data = mortgage_clean.merge(
    covenants_per_tract,
    on='census_tract',
    how='left'
)

# Fill NaN covenant counts with 0
combined_data['covenant_count'] = combined_data['covenant_count'].fillna(0)

# Basic Analysis
print("\nMerge Results:")
print("Total mortgage applications:", len(combined_data))
print("Applications in tracts with covenants:", (combined_data['covenant_count'] > 0).sum())
print("Unique tracts with covenants:", combined_data[combined_data['covenant_count'] > 0]['census_tract'].nunique())

# Calculate approval rates
combined_data['was_approved'] = combined_data['action_taken'].isin([1, 2])
covenant_areas = combined_data['covenant_count'] > 0

print("\nApproval Rates:")
print("Tracts with no covenants:", 
      combined_data[~covenant_areas]['was_approved'].mean())
print("Tracts with covenants:", 
      combined_data[covenant_areas]['was_approved'].mean())

Mortgage data examples:
0    27037060737
2    27141030504
3    27023950400
4    27171100101
5    27159480100
Name: census_tract, dtype: object

Covenant data examples:
Series([], Name: census_tract, dtype: object)

Merge Results:
Total mortgage applications: 100931
Applications in tracts with covenants: 0
Unique tracts with covenants: 0

Approval Rates:
Tracts with no covenants: 0.6646619968097017
Tracts with covenants: nan


In [20]:
combined_data.columns

Index(['activity_year', 'census_tract', 'derived_race', 'action_taken',
       'loan_amount', 'property_value', 'income', 'interest_rate',
       'tract_minority_population_percent', 'tract_to_msa_income_percentage',
       'denial_reason-1', 'covenant_count', 'was_approved'],
      dtype='object')

In [21]:
combined_data.head()

,activity_year,census_tract,derived_race,action_taken,loan_amount,property_value,income,interest_rate,tract_minority_population_percent,tract_to_msa_income_percentage,denial_reason-1,covenant_count,was_approved
0,2023,27037060737,Race Not Available,6,185000.0,195000,NaN,6.125,40.37,73.48,10,0.0,False
1,2023,27141030504,Race Not Available,6,375000.0,385000,NaN,6.625,10.64,121.26,10,0.0,False
2,2023,27023950400,Race Not Available,6,105000.0,105000,NaN,6.125,8.92,90.13,10,0.0,False
3,2023,27171100101,Race Not Available,6,285000.0,285000,NaN,6.75,13.27,104.37,10,0.0,False
4,2023,27159480100,Race Not Available,6,175000.0,175000,NaN,6.875,5.87,82.22,10,0.0,False


In [22]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100931 entries, 0 to 100930
Data columns (total 13 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   activity_year                      100931 non-null  int64  
 1   census_tract                       100931 non-null  object 
 2   derived_race                       100931 non-null  object 
 3   action_taken                       100931 non-null  int64  
 4   loan_amount                        100931 non-null  float64
 5   property_value                     88593 non-null   object 
 6   income                             88200 non-null   float64
 7   interest_rate                      83227 non-null   object 
 8   tract_minority_population_percent  100931 non-null  float64
 9   tract_to_msa_income_percentage     100931 non-null  float64
 10  denial_reason-1                    100931 non-null  int64  
 11  covenant_count                     1009

In [23]:
test_data = pd.read_csv('Data/mortgage_covenant_data.csv')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100931 entries, 0 to 100930
Data columns (total 14 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   activity_year                      100931 non-null  int64  
 1   census_tract                       99915 non-null   float64
 2   derived_race                       100931 non-null  object 
 3   action_taken                       100931 non-null  int64  
 4   loan_amount                        100931 non-null  float64
 5   property_value                     83491 non-null   float64
 6   income                             88200 non-null   float64
 7   interest_rate                      78125 non-null   float64
 8   tract_minority_population_percent  100931 non-null  float64
 9   tract_to_msa_income_percentage     100931 non-null  float64
 10  denial_reason-1                    100931 non-null  int64  
 11  covenant_count                     1009

In [27]:
bins = [0, 1, 51, 101, 201]
labels = ['Low (1-50)', 'Medium (51-100)', 'High (101-200)', 'Very High (>200)']
combined_data['covenant_density'] = pd.cut(combined_data['covenant_count'], bins=bins, labels=labels)

In [37]:
combined_data.sample(20)

,activity_year,census_tract,derived_race,action_taken,loan_amount,property_value,income,interest_rate,tract_minority_population_percent,tract_to_msa_income_percentage,denial_reason-1,covenant_count,was_approved,covenant_density
855,2023,27169670801,Race Not Available,6,245000.0,285000,71.0,6.625,6.27,114.15,10,0.0,False,NaN
92340,2023,27037060835,Black or African American,4,735000.0,NaN,412.0,NaN,16.16,117.65,10,0.0,False,NaN
9018,2023,27073180200,White,4,75000.0,NaN,58.0,NaN,3.39,100.77,10,0.0,False,NaN
38881,2023,27123031100,White,2,205000.0,285000,39.0,0.0,60.41,57.15,10,0.0,True,NaN
44154,2023,27137000600,White,1,235000.0,285000,120.0,7.75,8.09,138.82,10,0.0,True,NaN
73644,2023,27021960804,White,1,505000.0,635000,345.0,7.625,5.83,102.02,10,0.0,True,NaN
95258,2023,27123041002,Race Not Available,4,205000.0,NaN,70.0,NaN,34.64,92.68,10,0.0,False,NaN
15176,2023,27013170200,White,5,55000.0,NaN,61.0,NaN,12.94,104.59,10,0.0,False,NaN
5203,2023,27141030301,White,1,175000.0,375000,58.0,6.499,5.39,91.12,10,0.0,True,NaN
25713,2023,27003050228,Race Not Available,6,285000.0,295000.0,91.0,6.5,26.42,80.85,10,0.0,False,NaN
